In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time
import sklearn.datasets
import random
import re
import pandas as pd
import feather

In [4]:
def separate_dataset(trainset, ratio = 0.5):
    datastring = []
    datatarget = []
    for i in range(len(trainset.data)):
        data_ = trainset.data[i].split('\n')
        data_ = list(filter(None, data_))
        data_ = random.sample(data_, int(len(data_) * ratio))
        for n in range(len(data_)):
            data_[n] = clearstring(data_[n])
        datastring += data_
        for n in range(len(data_)):
            datatarget.append(trainset.target[i])
    return datastring, datatarget

def clearstring(string):
    string = re.sub('[^A-Za-z0-9 ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = ' '.join(string)
    return string.lower()

trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)

train = pd.DataFrame(trainset.data,columns=['sequence'])
label = pd.DataFrame(trainset.target,columns=['labels'])

In [8]:
num_words = 500000
column_name="word_seg"
word_seq_len = 1800
embedding_vector=200 

In [5]:
from keras.preprocessing import text,sequence
from gensim.models import Word2Vec
num_words = 500000
import os
import numpy as np
#词向量
def w2v_pad(df_train,col, maxlen_,victor_size):
    
    tokenizer = text.Tokenizer(num_words=num_words, lower=False,filters="")
    tokenizer.fit_on_texts(list(df_train[col].values))
    #padding sequence
    train_ = sequence.pad_sequences(tokenizer.texts_to_sequences(df_train[col].values), maxlen=maxlen_)
    
    word_index = tokenizer.word_index
    
    count = 0
    nb_words = len(word_index)
    print(nb_words)
    all_data=df_train[col]
    file_name = './Word2Vec_' + col  +"_"+ '200' + '.model'
    if not os.path.exists(file_name):
        #word2vec建模
        model = Word2Vec([[word for word in document.split(' ')] for document in all_data.values],
                         size=victor_size, window=5, iter=10, workers=11, seed=2018, min_count=2)
        model.save(file_name)
    else:
        model = Word2Vec.load(file_name)
    print("add word2vec finished....")    



    embedding_word2vec_matrix = np.zeros((nb_words + 1, victor_size))
    for word, i in word_index.items():
        embedding_vector = model[word] if word in model else None
        if embedding_vector is not None:
            count += 1
            embedding_word2vec_matrix[i] = embedding_vector
        else:
            unk_vec = np.random.random(victor_size) * 0.5
            unk_vec = unk_vec - unk_vec.mean()
            embedding_word2vec_matrix[i] = unk_vec

    
    return train_, word_index, embedding_word2vec_matrix

Using TensorFlow backend.


In [6]:
train_, word2idx, word_embedding = w2v_pad(train, 'sequence', 40, 200)

20465
add word2vec finished....


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [7]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

lb = LabelEncoder()
train_label = lb.fit_transform(label['labels'].values)
train_label = to_categorical(train_label)#变为one-hot向量

In [35]:
class Model:
    def __init__(self, size_layer, num_layers, embedded_matrix,
                  dimension_output, learning_rate):
        
        def cells(size, reuse=False):
            return tf.nn.rnn_cell.BasicRNNCell(size,reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.float32, [None, dimension_output])
        
        encoder_embeddings = tf.Variable(embedded_matrix)
#         encoder_embeddings = tf.Variable(tf.random_uniform([20465, 200], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        for n in range(num_layers):
            (out_fw, out_bw), (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = cells(size_layer // 2),
                cell_bw = cells(size_layer // 2),
                inputs = encoder_embedded,
                dtype = tf.float64,
                scope = 'bidirectional_rnn_%d'%(n))
            encoder_embedded = tf.concat((out_fw, out_bw), 2)
            
        W = tf.get_variable('w',shape=(128, dimension_output),dtype=tf.float64,initializer=tf.orthogonal_initializer())
        b = tf.get_variable('b',shape=(dimension_output),dtype=tf.float64,initializer=tf.zeros_initializer())
        self.logits = tf.matmul(encoder_embedded[:, -1], W) + b
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [36]:
size_layer = 128
num_layers = 1
embedded_size = 128
dimension_output = len(trainset.target_names)
learning_rate = 1e-3
maxlen = 50
batch_size = 128

In [37]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(size_layer,num_layers,word_embedding,dimension_output,learning_rate)
sess.run(tf.global_variables_initializer())

In [40]:
train_X, test_X, train_Y, test_Y = train_test_split(train_, train_label, test_size = 0.2)

In [41]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(0, (len(train_X) // batch_size) * batch_size, batch_size):
        batch_x = train_X[i:i+batch_size]
        batch_y = train_Y[i:i+batch_size]
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : batch_x, model.Y : batch_y})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_X) // batch_size) * batch_size, batch_size):
        batch_x = test_X[i:i+batch_size]
        batch_y = test_Y[i:i+batch_size]
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : batch_x, model.Y : batch_y})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_X) // batch_size)
    train_acc /= (len(train_X) // batch_size)
    test_loss /= (len(test_X) // batch_size)
    test_acc /= (len(test_X) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    EPOCH += 1

epoch: 0, pass acc: 0.000000, current acc: 0.612793
time taken: 3.5116209983825684
epoch: 0, training loss: 0.687362, training acc: 0.556108, valid loss: 0.653968, valid acc: 0.612793

epoch: 1, pass acc: 0.612793, current acc: 0.666992
time taken: 2.737776041030884
epoch: 1, training loss: 0.562533, training acc: 0.719223, valid loss: 0.632978, valid acc: 0.666992

time taken: 2.5582115650177
epoch: 2, training loss: 0.296201, training acc: 0.879261, valid loss: 0.796825, valid acc: 0.613281

time taken: 2.6869637966156006
epoch: 3, training loss: 0.123520, training acc: 0.955966, valid loss: 0.978774, valid acc: 0.615234

time taken: 2.6312808990478516
epoch: 4, training loss: 0.080930, training acc: 0.971117, valid loss: 1.037981, valid acc: 0.631348

time taken: 2.780440330505371
epoch: 5, training loss: 0.054813, training acc: 0.979877, valid loss: 1.305806, valid acc: 0.614258

time taken: 2.8112645149230957
epoch: 6, training loss: 0.047365, training acc: 0.984612, valid loss: 1